# Note

In this notebook, I show how to use eg.py.

*eg* module is built on top of [**xarray**](http://xarray.pydata.org/),
which brings N-dimensional variants of the core pandas data structures.  

# Importing some libraries

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from PyLHD import eg

In [3]:
import xarray as xr

# Load file from local

We prepare example eg files at 'PyLHD/testings/eg_example1d.txt'.

In [4]:
# Load
filename = '../testings/eg_example2d.txt'
example_data = eg.load(filename)

# Basic usage

The generated *'example_data'* is xarray.DataSet, this inherites all the methods and members.  

## Access to coordinate data

For example, to see the coordinate data,

In [5]:
print(example_data.coords)

Coordinates:
  * TIME     (TIME) float64 0.01 0.02 0.03 0.04 0.05
  * R        (R) float64 0.1 0.2 0.3


The *coords* is an OrderedDict.  
The dict like method can be used.

In [6]:
# to see all the keys.
example_data.coords.keys()

KeysView(Coordinates:
  * TIME     (TIME) float64 0.01 0.02 0.03 0.04 0.05
  * R        (R) float64 0.1 0.2 0.3)

The entries of the OrderedDict is xr.DataArray that can be used as np.array,

In [7]:
# to access the values in 'TIME' coords
example_data.coords['TIME']

<xarray.DataArray 'TIME' (TIME: 5)>
array([ 0.01,  0.02,  0.03,  0.04,  0.05])
Coordinates:
  * TIME     (TIME) float64 0.01 0.02 0.03 0.04 0.05
Attributes:
    Unit: s

In [8]:
# This acts as array-like.
np.sum(example_data.coords['TIME'])

<xarray.DataArray 'TIME' ()>
array(0.15000000000000002)

## Access to variable data

To see what data are contained,

In [9]:
example_data.data_vars.keys()

KeysView(Data variables:
    a        (TIME, R) float64 0.11 0.12 0.13 0.21 0.22 0.23 0.31 0.32 0.33 ...
    b        (TIME, R) float64 1.0 1.0 1.0 2.0 2.0 2.0 3.0 3.0 3.0 4.0 4.0 ...
    c        (TIME, R) float64 10.0 10.0 10.0 20.0 20.0 20.0 30.0 30.0 30.0 ...
    d        (TIME, R) float64 100.0 100.0 100.0 200.0 200.0 200.0 300.0 ...)

where 'a', 'b', 'c', 'd' are the name of the contained variables.  
To access a particular coordinate,

In [10]:
print(example_data.data_vars['a'])

<xarray.DataArray 'a' (TIME: 5, R: 3)>
array([[ 0.11,  0.12,  0.13],
       [ 0.21,  0.22,  0.23],
       [ 0.31,  0.32,  0.33],
       [ 0.41,  0.42,  0.43],
       [ 0.51,  0.52,  0.53]])
Coordinates:
  * TIME     (TIME) float64 0.01 0.02 0.03 0.04 0.05
  * R        (R) float64 0.1 0.2 0.3
Attributes:
    Unit: s


## Access to attributes

All the information in the header is stored as attributes.

In [11]:
example_data.attrs

OrderedDict([('NAME', 'example-2d'),
             ('ShotNo', 6115),
             ('Date', '1/2915:14'),
             ('comments',
              OrderedDict([('PHI', '3.5'),
                           ('PHIunit', "'portNO'"),
                           ('comments', "'Be thickness = 7.5 micro m'")]))])

Other attributes that connects to coordinates or variables are stored as their attributes,

In [12]:
example_data['TIME'].attrs

OrderedDict([('Unit', 's')])

In [13]:
example_data['c'].attrs

OrderedDict([('Unit', 'V')])

# Save

To save any data as *eg* file format, please use eg.dump(data, filename) method.

In [14]:
new_filename = 'tmp_file.txt'
eg.dump(example_data, new_filename)

In [15]:
# To check the identity, I will reload this file again.
example_data2 = eg.load(new_filename)
example_data == example_data2

<xarray.Dataset>
Dimensions:  (R: 3, TIME: 5)
Coordinates:
  * TIME     (TIME) float64 0.01 0.02 0.03 0.04 0.05
  * R        (R) float64 0.1 0.2 0.3
Data variables:
    a        (TIME, R) bool True True True True True True True True True ...
    b        (TIME, R) bool True True True True True True True True True ...
    c        (TIME, R) bool True True True True True True True True True ...
    d        (TIME, R) bool True True True True True True True True True ...

## Preparation for save

To save *eg* file format, we need some information other than just a values.  
Simplest way is to make a xarray.DataSet file with an appropriate coordinates and variables.

In [16]:
x_axis = np.linspace(0.0,1.0,10)
y_axis = np.linspace(0.0,1.0,11)
a = np.random.randn(len(x_axis), len(y_axis))

In [17]:
dump_data = xr.Dataset( data_vars = {'a':(('x','y'), a)}, 
                        coords={'x':x_axis, 'y': y_axis})

In [18]:
dump_data

<xarray.Dataset>
Dimensions:  (x: 10, y: 11)
Coordinates:
  * x        (x) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * y        (y) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0
Data variables:
    a        (x, y) float64 -1.563 -0.01093 -1.396 -0.7539 -0.7721 0.03514 ...

In [19]:
# It is usually important to link unit data with coordinates and values.
dump_data['a'].attrs['Unit'] = 'V'
dump_data['x'].attrs['Unit'] = 'm'
dump_data['y'].attrs['Unit'] = 's'

In [20]:
eg.dump(dump_data, new_filename, NAME='temp_diag', ShotNo=6000)

In [21]:
# To check dumped data, I will reload this file again.
example_data2 = eg.load(new_filename)
example_data2

<xarray.EGdata>
Dimensions:  (x: 10, y: 11)
Coordinates:
  * x        (x) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * y        (y) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0
Data variables:
    a        (x, y) float64 -1.563 -0.01093 -1.396 -0.7539 -0.7721 0.03514 ...
Attributes:
    NAME: temp_diag
    ShotNo: 6000
    Date: 2016/12/1315:42:32
    comments: OrderedDict()

In [22]:
example_data2['a']

<xarray.DataArray 'a' (x: 10, y: 11)>
array([[-1.562766  , -0.01092657, -1.395639  , -0.7539233 , -0.7721255 ,
         0.03514394,  0.1257069 , -1.933911  ,  0.4982711 ,  1.35267   ,
        -0.3206674 ],
       [ 1.724778  ,  0.0549739 ,  0.9507024 , -3.324699  , -0.5052582 ,
         0.1192041 ,  0.2828641 , -0.1978293 , -0.2525981 ,  0.6909911 ,
        -1.759871  ],
       [ 0.8069041 , -0.8631582 ,  0.7576942 , -0.6676276 ,  1.815476  ,
        -0.2426941 , -0.6901838 , -0.3289584 ,  0.3290385 ,  0.7334659 ,
        -1.906926  ],
       [ 0.7620053 , -0.1759518 ,  0.9133975 ,  0.03131901, -0.0080099 ,
        -1.308937  , -1.747693  ,  0.3069759 , -1.527917  , -2.203252  ,
        -0.6787131 ],
       [ 0.3426372 , -1.291217  , -1.04788   , -0.3390357 , -0.307747  ,
        -0.1843734 , -0.6645383 ,  2.9428    ,  1.923766  ,  0.8632298 ,
         0.9401448 ],
       [-0.0060025 , -0.3408198 , -0.3991794 , -1.418897  ,  0.6306461 ,
         0.5652755 , -0.8449487 , -0.3955003 ,  2

In [23]:
example_data2['y']

<xarray.DataArray 'y' (y: 11)>
array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])
Coordinates:
  * y        (y) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0
Attributes:
    Unit: s

### Remove files

In [24]:
import os
os.remove(new_filename)